In [1]:
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
from Invtsim import * 
import numpy as np
import pandas as pd
from scipy.stats import norm
import warnings
warnings.simplefilter("ignore")
from tqdm import tqdm

In [2]:
fcst    = pd.read_pickle(f"{path1}721fcsts_base2cohe.pkl")
fitt    = pd.read_pickle(f"{path1}721fitts_base2cohe.pkl")

ets     = fcst.iloc[:,4:]
lgb     = fcst.iloc[:,:4]

etsf    = fitt.iloc[:,4:]
lgbf    = fitt.iloc[:,:4]

test    = pd.read_pickle(f"{path1}721future_28.pkl").reset_index(drop=True)
truth   = test["y"]
history = pd.read_pickle(f"{path1}721past_1913.pkl")

ets_resid = pd.concat({ f"{etsf.columns[i]}": history['y'] - etsf.iloc[:,i] for i in range(4)},axis=1)
lgb_resid = pd.concat({ f"{lgbf.columns[i]}": history['y'] - lgbf.iloc[:,i] for i in range(4)},axis=1)

In [3]:
fitt

,lgb_base,lgb_bu,lgb_td,lgb_mint,ets_base,ets_bu,ets_td,ets_mint
0,10.427386,31.892026,10.427386,6.554925,4.361950,7.949644,4.361950,6.269733
1,6.017043,18.424095,6.017043,8.308634,5.869864,8.418413,5.869864,7.040606
2,2.831026,5.822718,2.831026,3.816622,7.009515,8.708042,7.009515,7.274240
3,3.167066,5.835477,3.167066,4.299868,7.721546,8.858576,7.721546,7.508198
4,3.511534,4.095703,3.511534,4.408462,8.318248,9.023244,8.318248,8.263259
...,...,...,...,...,...,...,...,...
81658313,0.125571,0.125571,0.125571,0.028646,0.079155,0.079155,0.079155,0.017801
81658314,0.134732,0.134732,0.134732,0.032594,0.078329,0.078329,0.078329,0.041732
81658315,0.139261,0.139261,0.139261,0.036976,0.077508,0.077508,0.077508,0.073077
81658316,0.150921,0.150921,0.150921,0.037568,0.076693,0.076693,0.076693,0.069677


## Verify:

In [4]:
'''
lp = {"l1" :(0,1),
      "l2" :(1,(1+3)),
      "l4" :((1+3),(1+3+3)),
      "l3" :((1+3+3),(1+3+3+10)),
      "l5" :((1+3+10+3),(1+3+10+3+7)),
      "l6" :((1+3+10+3+7),(1+3+10+3+7+9)),
      "l7" :((1+3+10+3+7+9),(1+3+10+3+7+9+21)),
      "l8" :((1+3+10+3+7+9+21),(1+3+10+3+7+9+21+30)),
      "l10" :((1+3+10+3+7+9+21+30),(1+3+10+3+7+9+21+30+3049)),
      "l9":((1+3+10+3+7+9+21+30+3049),(1+3+10+3+7+9+21+30+3049+70)),
      "l11":((1+3+10+3+7+9+21+30+70+3049),(1+3+10+3+7+9+21+30+70+3049+9147)),
      "l12":((1+3+10+3+7+9+21+30+70+3049+9147),(1+3+10+3+7+9+21+30+70+3049+9147+30490))}'''

lp = {"l10" :(0,3049),"l11":(3049,3049+9147),"l12":(3049+9147,3049+9147+30490)}

def check(lp:dict, df:pd.DataFrame, col:str = 'backlog', gap:int = 28):
    asl = []
    for i in lp.keys():
        bt = df[lp[i][0]*gap:lp[i][1]*gap]
        l1 = bt[col]
        if col == 'backlog':
            print(f"{len(l1[l1==0])/len(l1)}")
            asl.append(len(l1[l1==0])/len(l1))
        else:
            asl.append(l1.mean())
    return asl

In [5]:
def loop(fcst, truth, residual, NAME:str,
         gap1: int = 28, gap2:int = 1913,
         n = None, 
         df_list:list = [], 
         L_ = 3):
    if n == None:
        n = int(len(fcst)/gap1)
    for i in tqdm(range(n)):
        fcst_1  = fcst[i*gap1:(i+1)*gap1]
        truth_1 = truth[i*gap1:(i+1)*gap1]
        res_1   = residual[i*gap2:(i+1)*gap2]
        df = InvtSim(fcst = fcst_1, truth = truth_1, residual = res_1, name = NAME, L=L_)
        res = df.ob_all_t()
        df_list.append(res.reset_index(drop = True))
    #results = pd.concat(res)
    print("-----------DOWN!------------")
    return pd.concat(df_list).reset_index(drop=True)

In [6]:
df = loop(fcst = fcst['lgb_base'], truth = truth, residual = lgb_resid['lgb_base'], NAME = 'lgb_base', df_list = [],L_=1,n=1)
a = df[['cb_90','cb_95','cb_99']]#[['ot_90','ip_90','ot_95','ip_95','ot_99','ip_99']]#[['backlog_90','backlog_95','backlog_99']]#

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 284.30it/s]

-----------DOWN!------------


In [9]:
trto = pd.read_pickle(f"365TRactual_L{lead_time}.pkl")
trfo = pd.read_pickle(f"365TRfcst_L{lead_time}.pkl")

In [10]:
trto

,name,ot_90,ot_95,ot_99
0,tr_lgb_base,0.0,0.0,0.0
1,tr_lgb_base,33444.0,33444.0,33444.0
2,tr_lgb_base,29750.0,29750.0,29750.0
3,tr_lgb_base,28667.0,28667.0,28667.0
4,tr_lgb_base,30553.0,30553.0,30553.0
...,...,...,...,...
15636595,tr_ets_mint,0.0,0.0,0.0
15636596,tr_ets_mint,0.0,0.0,0.0
15636597,tr_ets_mint,0.0,0.0,0.0
15636598,tr_ets_mint,0.0,0.0,0.0


In [18]:
125092800/8/365

42840.0

## LEAD TIME 1:

In [14]:
Invt_df = []
nam_lgb = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint"]
nam_ets = [ "ets_base", "ets_bu", "ets_td", "ets_mint"]
lead_time = 1
#trto = pd.read_pickle(f"365TRactual_L{lead_time}.pkl")
#trfo = pd.read_pickle(f"365TRfcst_L{lead_time}.pkl")
for i in nam_lgb:
    df = loop(fcst = fcst[i], truth = truth, residual = lgb_resid[i], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"721lgbInvtSim_L{lead_time}.pkl")

Invt_df = []
for i in nam_ets:
    df = loop(fcst = fcst[i], truth = truth, residual = ets_resid[i], NAME = i, df_list = [],L_=lead_time)
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"721etsInvtSim_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [01:18<00:00, 545.46it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [01:19<00:00, 540.27it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [01:17<00:00, 549.42it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [01:17<00:00, 548.61it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42686/42686 [01:17<00:00, 551.25it/s]


-----------DOWN!------------


In [9]:
Invt_df = []
nam_lgb = [ "lgb_base", "lgb_bu", "lgb_td", "lgb_mint"]
nam_ets = [ "ets_base", "ets_bu", "ets_td", "ets_mint"]
lead_time = 14
for i in nam_lgb:
    df = loop(fcst = fcst[i], truth = truth, residual = lgb_resid[i], NAME = i, df_list = [],L_=lead_time)# L_=3
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"lgbInvtSim_L{lead_time}.pkl")

Invt_df = []
for i in nam_ets:
    df = loop(fcst = fcst[i], truth = truth, residual = ets_resid[i], NAME = i, df_list = [],L_=lead_time)
    Invt_df.append(df)
pd.concat(Invt_df).to_pickle(f"etsInvtSim_L{lead_time}.pkl")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:59<00:00, 358.85it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:58<00:00, 360.82it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:59<00:00, 357.53it/s]


...

-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:58<00:00, 360.87it/s]


-----------DOWN!------------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42840/42840 [01:57<00:00, 363.58it/s]


-----------DOWN!------------


## Inventory Reconciliation

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.simplefilter("ignore")
lgbsim = pd.read_pickle("lgbInvtSim_L1.pkl")
etssim = pd.read_pickle("etsInvtSim_L1.pkl")

In [2]:
lgbsim

,name,true_demand,forecasts,ot_90,sst_90,ip_90,net_90,backlog_90,ch_90,cb_90,...,backlog_95,ch_95,cb_95,ot_99,sst_99,ip_99,net_99,backlog_99,ch_99,cb_99
0,lgb_base,38793,37936.346099,0.000000,544.190211,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,131.192218,987.846119,131.192218,131.192218,0.000000,131.192218,0.000000
1,lgb_base,35487,35346.413219,35890.603430,544.190211,403.603430,403.603430,0.000000,403.603430,0.000000,...,0.00000,557.873794,0.000000,36203.067121,987.846119,847.259339,847.259339,0.000000,847.259339,0.000000
2,lgb_base,34445,33967.281969,34107.868750,544.190211,66.472180,66.472180,0.000000,66.472180,0.000000,...,0.00000,220.742543,0.000000,34107.868750,987.846119,510.128088,510.128088,0.000000,510.128088,0.000000
3,lgb_base,34732,34286.477860,34764.195891,544.190211,98.668071,98.668071,0.000000,98.668071,0.000000,...,0.00000,252.938435,0.000000,34764.195891,987.846119,542.323980,542.323980,0.000000,542.323980,0.000000
4,lgb_base,42896,38999.291696,39444.813836,544.190211,-3352.518093,-3352.518093,3352.518093,0.000000,30172.662835,...,3198.24773,0.000000,60766.706862,39444.813836,987.846119,-2908.862185,-2908.862185,2908.862185,0.000000,287977.356283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199515,lgb_mint,0,0.096519,0.000000,0.500212,0.791690,0.791690,0.000000,0.791690,0.000000,...,0.00000,0.933493,0.000000,0.000000,0.908014,1.199492,1.199492,0.000000,1.199492,0.000000
1199516,lgb_mint,0,0.113224,0.000000,0.500212,0.791690,0.791690,0.000000,0.791690,0.000000,...,0.00000,0.933493,0.000000,0.000000,0.908014,1.199492,1.199492,0.000000,1.199492,0.000000
1199517,lgb_mint,0,0.089249,0.000000,0.500212,0.791690,0.791690,0.000000,0.791690,0.000000,...,0.00000,0.933493,0.000000,0.000000,0.908014,1.199492,1.199492,0.000000,1.199492,0.000000
1199518,lgb_mint,0,0.145623,0.000000,0.500212,0.791690,0.791690,0.000000,0.791690,0.000000,...,0.00000,0.933493,0.000000,0.000000,0.908014,1.199492,1.199492,0.000000,1.199492,0.000000


In [8]:
lgbsim = pd.read_pickle("lgbInvtSim.pkl")

In [3]:
lgbsim['name'].unique()

array(['lgb_base', 'lgb_bu', 'lgb_td', 'lgb_mint'], dtype=object)